In [1]:
#@title Installing needed dependencies!
from IPython.display import clear_output
try:
    import flair
    from flair.data import Sentence
    from flair.models import SequenceTagger
    from tqdm import tqdm
    import os
    import sys
    import pandas as pd
    print("Modules required are already installed!")
except ModuleNotFoundError:
    %pip install flair
    clear_output()
    %pip install tqdm
    clear_output()
    %pip install pandas
    clear_output()
    print("All modules required were sucessfully installed!")

All modules required were sucessfully installed!


In [4]:
#@title Load model

import flair
from flair.data import Sentence
from flair.models import SequenceTagger
from tqdm import tqdm
import os
import sys
import pandas as pd

# Initialize Flair Sentiment Classifier
tagger = SequenceTagger.load("flair/ner-english-large")

2023-04-07 02:42:33,318 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [ ]:
#@title App Reviews with NER

#@markdown * ### Pick store:
store = "Google Play Store" #@param ["App Store", "Google Play Store"]
#@markdown * ### Pick app: 
app = "encode" #@param ["codeacademy", "datacamp", "encode", "learn-python-programiz", "mimo", "programming-hero", "programming-hub", "python-x", "sololearn"]

dir = "https://raw.githubusercontent.com/lopezbec/AI_Gamification_Python/main/reviews/review-files/"

if store == "App Store":
    input_file = "applestore-review-" + app + ".csv"
else:
    input_file = "googleplay-review-" + app + ".csv"

# Load the CSV file into a dataframe
df = pd.read_csv(dir + input_file)

output_df = pd.DataFrame(columns=['reviewId', 'NER_Text', 'Start_Pos', 'Eng_Pos', 'NER_Label', 'Prob'])

# iterate over each row of the input dataframe
for index, row in tqdm( df.iterrows()):
    # get the content and content_ID from the current row
    if store == "App Store":
      content = row['review']
    else:
      content = row['content']
    content_id = row[0]
    try: 
    # create a Flair sentence from the content
        sentence = Sentence(content)
   # print(sentence)
    # run NER on the sentence
        tagger.predict(sentence)
    except:
        continue
    
    # iterate over each entity in the sentence and add it to the output dataframe
    for entity in sentence.get_spans('ner'):
        
        new_row = pd.Series({
            'reviewId': content_id,
            'NER_Text': entity.text,
            'Start_Pos': entity.start_position,
            'End_Pos': entity.end_position,
            'NER_Label': entity.tag,
            'Prob': entity.score})

        output_df = pd.concat([output_df, new_row.to_frame().T], ignore_index=True)

#@markdown * ### Display data?
visualize = "Yes" #@param ["Yes", "No"]
if visualize == "Yes":
    display(output_df)
else:
    print("No display")
#@markdown * ### Save to csv?
csv = "Yes" #@param ["Yes", "No"]
if csv == "Yes":
    # Get the output file name by appending "_sentiment" before the file extension
    output_file = os.path.splitext(input_file)[0] + "_NER.csv"
    # Save the new CSV file with sentiment column
    output_df.to_csv(output_file, index=False)
else:
    print("No export")

59it [00:31,  1.53it/s]